In [1]:
import keras
import pickle
import random
import numpy as np
import pandas as pd
import re
import multiprocessing
import os
import itertools

from tqdm import *
from keras.models import Sequential, Model
from keras.layers import Dense, Embedding, Input, Reshape, concatenate, Flatten, Activation, LSTM, Dropout, Lambda
from keras.utils import np_utils

Using TensorFlow backend.


In [5]:
##### CONFIGURATION SETUP ####
data_path = "../logs/not_normalized/bpic2011.xes"
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
target_variable = "concept:name"
### CONFIGURATION SETUP END ###

In [3]:
def load_trace_dataset(purpose='categorical', ttype='test'):
    suffix = "_{0}_{1}.pickled".format(purpose, ttype)
    p = data_path.replace(".xes", suffix)
    return pickle.load(open(p, "rb"))

In [6]:
train_traces_categorical = load_trace_dataset('categorical', 'train')
train_traces_ordinal = load_trace_dataset('ordinal', 'train')
train_traces_targets = load_trace_dataset('target', 'train')
train_traces_sp2 = load_trace_dataset('sp2', 'train')

feature_dict = load_trace_dataset('mapping', 'dict')

## Data preparation

In [7]:
# Use one-hot encoding for categorical values
for col in train_traces_categorical[0].columns:
    nc = len(feature_dict[col]['to_int'].values())
    for i in range(0, len(train_traces_categorical)):
        tmp = train_traces_categorical[i][col].map(feature_dict[col]['to_int'])
        tmp = np_utils.to_categorical(tmp, num_classes=nc)
        tmp = pd.DataFrame(tmp).add_prefix(col)
        
        train_traces_categorical[i].drop(columns=[col], inplace=True)
        train_traces_categorical[i] = pd.concat([train_traces_categorical[i], tmp], axis=1)

In [8]:
train_traces = [ pd.concat([a,b], axis=1) for a,b in zip(train_traces_ordinal, train_traces_categorical)]

In [9]:
n_train_cols  = len(train_traces[0].columns)
n_target_cols = len(train_traces_targets[0].columns)
window_size = 5
train_inputs = []

for t in train_traces:   
    for j in range(0, len(t)-window_size):
        train_inputs.append(t.values[j:j+window_size])

# drop the last few inputs because that's when stuff ends, taking into account the EOS marker
# drop the first few targets because of the window size
train_inputs = np.array(train_inputs, dtype=np.float32)
train_targets = np.concatenate([ t[window_size:] for t in train_traces_targets ])
assert(len(train_inputs) == len(train_targets))

## Model creation

In [19]:
batch_size = None # None translates to unknown batch size
output_count = len(feature_dict[target_variable]["to_int"])
unit_count = n_train_cols + output_count

# [samples, time steps, features]
il = Input(batch_shape=(batch_size,None,n_train_cols))

main_output = LSTM(unit_count,
                   batch_input_shape=(batch_size,None,n_train_cols),
                   stateful=False,
                   return_sequences=True,
                   dropout=0.3)(il)
main_output = LSTM(unit_count,
                   stateful=False,
                   return_sequences=False,
                   dropout=0.3)(main_output)

main_output = Dense(output_count, activation='softmax', name='dense_final')(main_output)
full_model = Model(inputs=[il], outputs=[main_output])
optimizerator = keras.optimizers.RMSprop()

full_model.compile(loss='categorical_crossentropy', optimizer=optimizerator, metrics=['accuracy'])

In [17]:
%matplotlib inline

import keras
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Flatten, Dense, Activation

import numpy as np
from matplotlib import pyplot as plt
from IPython.display import clear_output
class PlotLosses(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []
        
        self.fig = plt.figure()
        
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.i += 1
        
        clear_output(wait=True)
        plt.plot(self.x, self.losses, label="loss")
        plt.plot(self.x, self.val_losses, label="val_loss")
        plt.legend()
        plt.show();
        
plot_losses = PlotLosses()

## Model training

In [20]:
history = full_model.fit(x=train_inputs,
               y=train_targets,
               batch_size=100,
               epochs=100,
               verbose=2,
               callbacks=[plot_losses],
               validation_split=0.3,
               shuffle=False)

Train on 75067 samples, validate on 32172 samples
Epoch 1/100


KeyboardInterrupt: 

<Figure size 432x288 with 0 Axes>